# MeTRAbsによる３次元解析（静止画版）

## ライブラリのインストール

In [1]:
#TensorFlow
import tensorflow as tf

In [2]:
#OpenCV
import cv2

In [3]:
#Matplotlib
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation as animation#アニメーション用

In [4]:
#データ処理用
import pickle
import numpy as np
import os

## モデルファイルのダウンロード（以下の2つのうちどちらかを実行）


google colab.の場合は以下を実行


In [ ]:
def download_model(model_type):
    server_prefix = 'https://omnomnom.vision.rwth-aachen.de/data/metrabs'
    model_zippath = tf.keras.utils.get_file(
        origin=f'{server_prefix}/{model_type}_20211019.zip',
        extract=True, cache_subdir='models')
    model_path = os.path.join(os.path.dirname(model_zippath), model_type)
    return model_path

model = tf.saved_model.load(download_model('metrabs_mob3l_y4t')) # or metrabs_eff2l_y4 for the big model

Anacondaの場合は以下を実行

https://github.com/isarandi/metrabs/blob/master/docs/MODELS.md
から対象のファイル（metrabs_mob3l_y4t）をダウンロードして展開したフォルダをmetrabs_movie.ipynbと同じフォルダに置く




In [5]:
model_path = os.path.join(os.path.dirname('./'), 'metrabs_mob3l_y4t')
model = tf.saved_model.load(model_path) # or metrabs_eff2l_y4 for the big model

## 可視化用

In [6]:
skt = []

def visualize_movie(im, detections, poses3d, poses2d, edges):
    global skt
    while len(skt)>0:
        skt.pop(0).remove()
    s = image_ax.imshow(im)
    skt.append(s)
    for x, y, w, h in detections[:, :4]:
        s = image_ax.add_patch(Rectangle((x, y), w, h, fill=False))
        skt.append(s)

    # Matplotlibではz軸が上方向ですが、MeTRabsではy方向が上方向です。そこで、x軸周りに90度回転させています。
    poses3d[..., 1], poses3d[..., 2] = poses3d[..., 2], -poses3d[..., 1]
    for pose3d, pose2d in zip(poses3d, poses2d):
        for i_start, i_end in edges:
            s = image_ax.plot(*zip(pose2d[i_start], pose2d[i_end]), marker='o', markersize=2)
            skt.append(s[0])
            s = pose_ax.plot(*zip(pose3d[i_start], pose3d[i_end]), marker='o', markersize=2)
            skt.append(s[0])


## 読み込むファイル名の設定

In [7]:
InputName = 'd_punch1'

## 3次元解析とデータの保存

In [8]:
image = tf.image.decode_jpeg(tf.io.read_file(InputName+'.jpg'))

savepose = []

pred = model.detect_poses(image, skeleton='smpl_24')
savepose.append([pred['boxes'].numpy(),pred['poses3d'].numpy(),pred['poses2d'].numpy()])

with open(InputName+'.pickle', mode='wb') as f:
    pickle.dump(savepose, f)

## データの読み込みと静止画の書き出し

In [9]:
fig = plt.figure(figsize=(10, 5.2))
image_ax = fig.add_subplot(1, 2, 1)
pose_ax = fig.add_subplot(1, 2, 2, projection='3d')
pose_ax.view_init(5, -85)
pose_ax.set_xlim3d(-1500, 1500)
pose_ax.set_zlim3d(-1500, 1500)
pose_ax.set_ylim3d(0, 3000)

image = tf.image.decode_jpeg(tf.io.read_file(InputName+'.jpg'))

loadpose = []
with open(InputName+'.pickle', mode='rb') as f:
    loadpose = pickle.load(f)

visualize_movie(
    image, #画像
    loadpose[0][0], #囲み線
    loadpose[0][1], #3次元位置
    loadpose[0][2], #2次元位置
    model.per_skeleton_joint_edges['smpl_24'].numpy()) 